In [42]:
# Set parameters for testing
# This code cell will be replaced by Times Square
# /sdf/home/h/huanlin/shared_campaigns_LSST/DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195
#submitHead = "/sdf/data/rubin/shared/campaigns/LSSTCam-Nightly-Validation-CC/"
#tasksHead = "DM-50157_20250609/submit/LSSTCam/runs/nightlyValidation/20250609/d_2025_06_09/DM-50157"
submitHead = "/sdf/home/h/huanlin/shared_campaigns_LSST/"
tasksHead = "DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195/"

In [43]:
# --------------------------------------------------------------------------------------
# failureLister - get list of failed task jobs and show the errors
# author: Homer Neal
# ---------------------------------------------------------------------------------------
import sys
import numpy as np
import time
import subprocess
import re
import math
import glob, os
import pandas as pd
from tabulate import tabulate
from IPython.display import display, HTML

# ---- types
#analyzeSingleVisitStar
#analyzeSingleVisitStarAssociation
#analyzeSingleVisitStarAstrometricRefMatch
#analyzeSingleVisitStarPhotometricRefMatch
#associateIsolatedStar
#coadd
#consolidateHealSparsePropertyMaps
#consolidateSingleVisitStar
#consolidateVisitSummary
#makeAnalysisSingleVisitStarAstrometricRefMatch
#makeAnalysisSingleVisitStarPhotometricRefMatch
#makeHealSparsePropertyMaps
#makeWarpTract
#selectDeepCoaddVisits

jobs = ["generateEphemerides","fgcmFitCycle","analyzeSingleVisitStar","analyzeSingleVisitStarAssociation","analyzeSingleVisitStarAstrometricRefMatch","analyzeSingleVisitStarPhotometricRefMatch","associateIsolatedStar","coadd","consolidateHealSparsePropertyMaps","consolidateSingleVisitStar","consolidateVisitSummary","finalJob","makeAnalysisSingleVisitStarAssociationMetricTable","makeAnalysisSingleVisitStarAssociationWholeSkyPlot","makeAnalysisSingleVisitStarAstrometricRefMatch","makeAnalysisSingleVisitStarPhotometricRefMatch","makeHealSparsePropertyMaps","makeInitialVisitDetectorTable","makeInitialVisitTable","makeWarpTract","pipetaskInit","plotPropertyMapSurvey","selectDeepCoaddVisits","step1detector"]
#jobs = ["analyzeSingleVisitStarAssociation"]
df = pd.DataFrame(columns=['job name','errors'])

def dumpError(flin):
    err = ""
    fpin = open(flin)
    for ln in fpin:
        if "ERROR" in ln.upper() :
            err = ln
            break
    return err

# tasksHead = "LSSTCam/runs/nightlyValidation/20250530/d_2025_05_30/DM-50157/"
# tasksHead = "DM-50157_20250607/submit/LSSTCam/runs/nightlyValidation/20250607/d_2025_06_08/DM-50157/"

os.chdir(submitHead+tasksHead)
# nodeStatFile=sys.argv[1]


FileNotFoundError: [Errno 2] No such file or directory: '/sdf/home/h/huanlin/shared_campaigns_LSST/DM-53195/submit/LSSTCam/runs/DRP/20250421_20250921/d_2025_11_10/DM-53195/'

In [40]:
for nodeStatFile in glob.glob("*/*.node_status"):
#nodeStatFile = submitHead+tasksHead+"20250531T113935Z/LSSTCam_runs_nightlyValidation_20250530_d_2025_05_30_DM-50157_20250531T113935Z.node_status"
#nodeStatFile = input("Path of node status file = ")
#    ./20250611T154923Z/jobs/analyzeSingleVisitStar/22e500cc-c984-400a-be69-d182b405bd9d_analyzeSingleVisitStar_2025060900506_.17979981.err
#      Type = "NodeStatus";
#  Node = "22e500cc-c984-400a-be69-d182b405bd9d_analyzeSingleVisitStar_2025060900506_";
#  NodeStatus = 5; /* "STATUS_DONE" */

#    ./20250611T154923Z/jobs/analyzeSingleVisitStarAssociation/040f8daf-bfbd-4574-ba7f-02219bc3fb41_analyzeSingleVisitStarAssociation_7041_.18061387.err
    print("node_status file = "+nodeStatFile)
    fp = open(nodeStatFile)
    nextline = False
    for ln in fp:
        if ("Type" in ln and "NodeStatus" in ln):
    #        print(ln)
            for ln2 in fp:
                if ("]" in ln2):
                    break
                #print(ln2.strip().split()[0])
                if (ln2.strip().split()[0] == "Node"):
                    element = ln2.strip().split()[2].strip(';').strip('"')
#                    print("element = "+element)
                if (ln2.strip().split()[0] == "NodeStatus"):
                    if ("ERROR" in ln2):
                        element_pieces = element.split("_")
                        element_2_pieces = element.split("-")
                        fifthElement = ""
                        try:
                            fifthElement = element_pieces[1]
                            lower_path = ""
                            if len(element_pieces)>2 :
                                for ielem in element_pieces[2:] :
                                    lower_path = lower_path + "/" + ielem
#                            print("lower_path = "+lower_path+" len = "+str(len(lower_path)))
#                            fifthElement = element_2_pieces[5]
                        except:
                            pass
                        flerr = ""
                        for jb in jobs:
#                            print("jb = "+jb+" | element_pieces[0] = "+element_pieces[0]+" | fifthElement = "+fifthElement)
#                            print("step1")
                            if jb in element_pieces[0] or nextline:
                                try:
                                    flerr = glob.glob("*/jobs/"+jb+"/"+element_pieces[1]+"/"+jb+"_"+element_pieces[1]+"_"+element_pieces[2]+".*.err")[0]
                                    new_row = pd.DataFrame({"job name": [element_pieces[0]], "errors":[dumpError(flerr)]})
                                    df = pd.concat([df, new_row], ignore_index=True)
                                except:
                                    pass
#                            print("step2")
                            if jb==fifthElement or nextline:
#                                print("jb = "+jb)
                                try:
                                    if len(lower_path)<1 :
                                        flerr = glob.glob("*/jobs/"+jb+"/"+element+"*.err")[0]
                                    else:
#                                        print("trying - "+"*/jobs/"+jb+lower_path+element+"*.*")
                                        new_row = None
                                        for flerr in glob.glob("*/jobs/"+jb+lower_path+element+"*.*"):
#                                            print("flerr = "+flerr)
                                            try:
                                                the_error = dumpError(flerr)
                                                if len(the_error) > 0 :
                                                    new_row = pd.DataFrame({"job name": [fifthElement], "errors":[dumpError(flerr)]})
                                                    df = pd.concat([df, new_row], ignore_index=True)
                                            except:
                                                pass
                                except:
                                    pass
#                            print(element_pieces[1]+"/"+element_pieces[2])
#                            print("*/jobs/step1detector/"+element_pieces[1]+"/step1detector_"+element_pieces[1]+"_"+element_pieces[2]+".*.err")
                        
##                                print(element + ": stat = " + ln2.strip().split()[4].strip('"') + " file = " + flerr )
##                                print(dumpError(flerr))
                        if not nextline:
                            nextline = False # deactivating
                        else:
                            nextline = False
#    print(df)
#    print(tabulate(df, headers='keys', tablefmt='psql'))

    display(HTML(df.to_html()))
    

node_status file = 20250611T154923Z/LSSTCam_runs_nightlyValidation_20250609_d_2025_06_09_DM-50157_20250611T154923Z.node_status
lower_path = /2025060900633/ len = 15
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = 904cab00-2fa5-48c0-99b4-2273f5ff78fc | fifthElement = analyzeSingleVisitStar
step1
step2
lower_path = /2025060900363/ len = 15
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = e345a3f8-5a61-49e8-8d45-64f3d148a626 | fifthElement = analyzeSingleVisitStar
step1
step2
lower_path = /2025060900383/ len = 15
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = aa20339d-2186-4083-80e0-e305198d3b99 | fifthElement = analyzeSingleVisitStar
step1
step2
lower_path = /2025060900432/ len = 15
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = 5dc78491-1b1f-4e84-8ab4-63272d187eb8 | fifthElement = analyzeSingleVisitStar
step1
step2
lower_path = /2025060900168/ len = 15
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = be560f0f-b74e-4a1e-81e

,job name,errors


node_status file = 20250610T112746Z/LSSTCam_runs_nightlyValidation_20250609_d_2025_06_09_DM-50157_20250610T112746Z.node_status
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 70
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 119
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 118
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 111
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 48
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | element_pieces[0] = step1detector | fifthElement = 91
step1
step2
lower_path = /001 len = 4
jb = analyzeSingleVisitStarAssociation | eleme

,job name,errors
